In [2]:
from my_transformers import ClassificationTransformer

import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F

from torchtext import datasets, vocab
from transformers import AutoTokenizer
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader

import numpy as np

from torch.utils.tensorboard import SummaryWriter

import random, tqdm, sys, math, gzip

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:



tbw = SummaryWriter()
batch_size = 32



# train, test = datasets.IMDB()

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

dataset = load_dataset("imdb")

Found cached dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
100%|███████████████████████████████████████████| 3/3 [00:00<00:00, 289.80it/s]


In [15]:

print(dataset)
dataset = dataset.map(lambda x: tokenizer(x['text'], padding='max_length', truncation=True, max_length=100, return_tensors='pt'), batched=True)
dataset = dataset.map(lambda x: {"tokens": x['input_ids']},batched=True)
dataset = dataset.map(lambda x: {"labels": x['label']}, batched=True)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})


In [19]:
# dataset.train_test_split(test_size=0.1)
dataset.set_format(type="torch", columns=["tokens", "labels"])
# dataset = load_dataset("imdb")
# train = list(train)
# test = list(test)
# train = torch.utils.data.DataLoader(train, batch_size=batch_size, drop_last=True)
# test = torch.utils.data.DataLoader(test, batch_size=batch_size, drop_last=True)
mx = 1000


batch_size = 32
train_loader = DataLoader(dataset['train'], batch_size=batch_size, shuffle=True)
test_laoder = DataLoader(dataset['test'], batch_size=batch_size, shuffle=True)

print(next(iter(train_loader)))

# def tokenize_data(data, tokenizer, max_length=30):
    
#     tokenized_data = []
#     for label, text in tqdm.tqdm(data):
#         tokens = tokenizer(text, padding='max_length', truncation=True, max_length=max_length, return_tensors='pt')
#         tokenized_data.append((label, tokens['input_ids'].squeeze(0)))
#     return tokenized_data

# train = tokenize_data(train, tokenizer)
# test = tokenize_data(test, tokenizer)

{'tokens': tensor([[  101,  1045,  2293,  ...,  2019,  1055,   102],
        [  101,  2023,  2524,  ...,  2095,  1010,   102],
        [  101,  2007,  1005,  ...,  2040,  2763,   102],
        ...,
        [  101,  4212,  2046,  ...,  2004,  2045,   102],
        [  101,  2043,  1045,  ...,  2191,  7078,   102],
        [  101,  1045,  1005,  ...,  4626, 21862,   102]]), 'labels': tensor([0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1,
        1, 0, 1, 0, 1, 0, 0, 1])}


In [4]:
# %env CUDA_LAUNCH_BLOCKING=1

In [68]:
device='cpu'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import importlib
import my_transformers
importlib.reload(my_transformers)





model = my_transformers.ClassificationTransformer(k=4, heads=4, depth=2, seq_length=mx, num_tokens=tokenizer.vocab_size, num_classes=2)
model = model.to(device)
opt = torch.optim.Adam(lr=0.001, params=model.parameters())
criterion = nn.NLLLoss()

model.train()
epochs = 50
losses = []

for _ in tqdm.tqdm(range(epochs)):
    for loader in train_loader:
        # print(loader['labels'], loader['tokens'])
        labels, tokens = loader['labels'].to(device), loader['tokens'].to(device)
        # labels -= 1
        
        opt.zero_grad()
        out = model(tokens)
        loss = criterion(out, labels)
        loss.backward()
        opt.step()

    losses.append(loss.item())

print(losses)


100%|██████████████████████████████████████████| 50/50 [08:59<00:00, 10.78s/it]

[0.6281536817550659, 0.3914911448955536, 0.2477555125951767, 0.44476237893104553, 0.5029935836791992, 0.23210160434246063, 0.2422540932893753, 0.10572236031293869, 0.44623589515686035, 0.377618670463562, 0.4931173324584961, 0.2590349614620209, 0.619849681854248, 0.30617913603782654, 0.0948827937245369, 0.13597002625465393, 0.2991216778755188, 0.3802453577518463, 0.33251169323921204, 0.16540858149528503, 0.0673525407910347, 0.025242848321795464, 0.015696093440055847, 0.0271279439330101, 0.04158104211091995, 0.00987352803349495, 0.005758997518569231, 0.009109395556151867, 0.0051191020756959915, 0.6534697413444519, 0.04203469678759575, 0.004979562945663929, 0.006617422681301832, 0.004884060472249985, 0.015144217759370804, 0.0028130970895290375, 0.002308955416083336, 0.021797845140099525, 0.005704130977392197, 0.008588522672653198, 0.0018409698968753219, 0.03512774035334587, 0.0011812576558440924, 0.0023488670121878386, 0.004884304478764534, 0.005404841620475054, 0.0014519558753818274, 0.0

In [69]:
print(losses)

[0.6281536817550659, 0.3914911448955536, 0.2477555125951767, 0.44476237893104553, 0.5029935836791992, 0.23210160434246063, 0.2422540932893753, 0.10572236031293869, 0.44623589515686035, 0.377618670463562, 0.4931173324584961, 0.2590349614620209, 0.619849681854248, 0.30617913603782654, 0.0948827937245369, 0.13597002625465393, 0.2991216778755188, 0.3802453577518463, 0.33251169323921204, 0.16540858149528503, 0.0673525407910347, 0.025242848321795464, 0.015696093440055847, 0.0271279439330101, 0.04158104211091995, 0.00987352803349495, 0.005758997518569231, 0.009109395556151867, 0.0051191020756959915, 0.6534697413444519, 0.04203469678759575, 0.004979562945663929, 0.006617422681301832, 0.004884060472249985, 0.015144217759370804, 0.0028130970895290375, 0.002308955416083336, 0.021797845140099525, 0.005704130977392197, 0.008588522672653198, 0.0018409698968753219, 0.03512774035334587, 0.0011812576558440924, 0.0023488670121878386, 0.004884304478764534, 0.005404841620475054, 0.0014519558753818274, 0.0

In [79]:
with torch.no_grad():
    total_pred = 0
    total_correct = 0

    for loader in test_laoder:
        labels, tokens = loader['labels'].to(device), loader['tokens'].to(device)
        out = model(tokens)
        _, pred = torch.max(out, 1)
        total_pred += labels.shape[0]

        total_correct += (pred == labels).sum().item()

        # i = 30
        # print(tokenizer.decode(tokens[i]))
        # print(labels[i])
        # 0 neg

print(total_correct/total_pred)

0.73908


In [78]:

review_text = "Toy story is a movie, or is it? "
with torch.no_grad():
    review_token = tokenizer(review_text, padding='max_length', truncation=True, max_length=100, return_tensors='pt')['input_ids'][0]
    review_token = review_token.to(device)
    out = model(review_token.view(1, -1))
    _, pred = torch.max(out, 1)

if pred.item() == 0:
    print('negative')
else:
    print('positive')


positive
